<a href="https://colab.research.google.com/github/Nastia12345678/PAS/blob/main/traceback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **traceback** - Исключения и трассировки стека

**Traceback** (трассировка) — это отчет, который содержит вызовы выполненных функций в вашем коде в определенный момент.


# Вспомогательные функции

В примерах в этом разделе используется модуль traceback_example.py.

In [3]:
#traceback_example.py 
import traceback
import sys


def produce_exception(recursion_level=2):
    sys.stdout.flush()
    if recursion_level:
        produce_exception(recursion_level - 1)
    else:
        raise RuntimeError()


def call_function(f, recursion_level=2):
    if recursion_level:
        return call_function(f, recursion_level - 1)
    else:
        return f()

# Изучение стека

**format()** Метод создает последовательность форматированных строк готовую для печати.

In [ ]:
#traceback_stacksummary.py 
import traceback
import sys


def f():
    summary = traceback.StackSummary.extract(
        traceback.walk_stack(None)
    )
    print(''.join(summary.format()))


print('Calling f() directly:')
f()

print()
print('Calling f() from 3 levels deep:')
call_function(f)

Это StackSummary итеративный контейнер, содержащий FrameSummary экземпляры.

In [ ]:
#traceback_framesummary.py 
import traceback
import sys

template = (
    '{fs.filename:<26}:{fs.lineno}:{fs.name}:\n'
    '    {fs.line}'
)


def f():
    summary = traceback.StackSummary.extract(
        traceback.walk_stack(None)
    )
    for fs in summary:
        print(template.format(fs=fs))


print('Calling f() directly:')
f()

print()
print('Calling f() from 3 levels deep:')
call_function(f)


# **TracebackException**

Класс TracebackException — это высокоуровневый интерфейс для создания
объектов StackSummary в процессе обработки трассировочной информации.

In [ ]:
#traceback_tracebackexception.py 
import traceback
import sys

print('with no exception:')
exc_type, exc_value, exc_tb = sys.exc_info()
tbe = traceback.TracebackException(exc_type, exc_value, exc_tb)
print(''.join(tbe.format()))

print('\nwith exception:')
try:
    produce_exception()
except Exception as err:
    exc_type, exc_value, exc_tb = sys.exc_info()
    tbe = traceback.TracebackException(
        exc_type, exc_value, exc_tb,
    )
    print(''.join(tbe.format()))

    print('\nexception only:')
    print(''.join(tbe.format_exception_only()))

**format()** Метод производит отформатированный вариант полного **TRACEBACK**, а **format_exception_only()** показывает только сообщение об исключении.

# **Низкоуровневые программные интерфейсы исключений**


Еще один способ обработки сообщений об исключениях предлагает функция
**print_exc()**. Она использует функцию **sys.exc__info()** для получения информации об исключениях, возникших в текущем потоке, форматирует результаты
и выводит текст в файл, заданный дескриптором.

In [ ]:
#traceback_print_exc.py 
import traceback
import sys

print('print_exc() with no exception:')
traceback.print_exc(file=sys.stdout)
print()

try:
    produce_exception()
except Exception as err:
    print('print_exc():')
    traceback.print_exc(file=sys.stdout)
    print()
    print('print_exc(1):')
    traceback.print_exc(limit=1, file=sys.stdout)

В этом примере дескриптор файла **sys.stdout** заменяется так, чтобы информационные сообщения и сообщения трассировки корректно смешивались.

**print_exc()** это просто ярлык для **print_exception()**, который требует явных аргументов.

In [ ]:
#traceback_print_exception.py 
import traceback
import sys

try:
    produce_exception()
except Exception as err:
    print('print_exception():')
    exc_type, exc_value, exc_tb = sys.exc_info()
    traceback.print_exception(exc_type, exc_value, exc_tb)

Аргументы для функции **print_exception()** получают c помощью функции
**sys.exc_info()**.

Для подготовки текста к выводу функция **print_exception()** использует
функцию **format_exception()**.

In [ ]:
#traceback_format_exception.py 
import traceback
import sys
from pprint import pprint

try:
    produce_exception()
except Exception as err:
    print('format_exception():')
    exc_type, exc_value, exc_tb = sys.exc_info()
    pprint(
        traceback.format_exception(exc_type, exc_value, exc_tb),
        width=65,
    )

Функции **format_exception ()** передаются те же три аргумента: тип исключения (exc_type), значение исключения (exc_value) и трассировочная информация (exc_tb).

Чтобы обработать трассировочную информацию иным способом, например
применить другое форматирование, следует использовать функцию **extract_
tb ()**, позволяющую извлечь данные в подходящем виде.

In [ ]:
#traceback_extract_tb.py 
import traceback
import sys
import os

template = '{filename:<23}:{linenum}:{funcname}:\n    {source}'

try:
    produce_exception()
except Exception as err:
    print('format_exception():')
    exc_type, exc_value, exc_tb = sys.exc_info()
    for tb_info in traceback.extract_tb(exc_tb):
        filename, linenum, funcname, source = tb_info
        if funcname != '<module>':
            funcname = funcname + '()'
        print(template.format(
            filename=os.path.basename(filename),
            linenum=linenum,
            source=source,
            funcname=funcname)
        )

Возвращаемое значение — это список записей, которые соответствуют различным уровням стека, представленного объектом трассировки. Каждая запись представляет собой кортеж, включающий четыре элемента: имя исходного файла,
номер строки в этом файле, имя функции и исходный код, находящийся в этой
строке, c исключенными пробелами (если он доступен).

# **Низкоуровневые программные интерфейсы стека**

Аналогичный набор функций доступен для выполнения тех же операций
по отношению к текущему стеку вызовов, а не к трассировочной информации.
Функция **print_stack ()** выводит текущий стек без возбуждения исключения.

In [ ]:
#traceback_print_stack.py 
import traceback
import sys

def f():
    traceback.print_stack(file=sys.stdout)


print('Calling f() directly:')
f()

print()
print('Calling f() from 3 levels deep:')
call_function(f)

Результат выглядит как трассировка без сообщения об ошибке.

Функция **format_stack ()** подготавливает информацию о стеке вызовов точно
так же, как функция **format exception ()** подготавливает к выводу трассировочную информацию.

In [ ]:
#traceback_format_stack.py 
import traceback
import sys
from pprint import pprint

def f():
    return traceback.format_stack()


formatted_stack = call_function(f)
pprint(formatted_stack)

Он возвращает список строк, каждая из которых составляет одну строку вывода.

**extract_stack()** Функция работает как **extract_tb()**.

In [ ]:
#traceback_extract_stack.py 
import traceback
import sys
import os

template = '{filename:<26}:{linenum}:{funcname}:\n    {source}'


def f():
    return traceback.extract_stack()


stack = call_function(f)
for filename, linenum, funcname, source in stack:
    if funcname != '<module>':
        funcname = funcname + '()'
    print(template.format(
        filename=os.path.basename(filename),
        linenum=linenum,
        source=source,
        funcname=funcname)
    )

# **Задание:** Вывести отчет об ошибки в любой своей программе с помощью Traceback.